# **Description**

Create a system that generates comic strips from story prompts using image generation models and panel layout algorithms.

Import necessary Modules.

In [22]:
!pip install diffusers transformers accelerate torch torchvision
!pip install nltk pillow matplotlib


Libraries

In [23]:
from nltk.tokenize import sent_tokenize
import os
from PIL import Image, ImageDraw, ImageFont
import torch
from diffusers import StableDiffusionPipeline

In [24]:
story = """
Alex walks into the café and sees Lily.
Alex: Hey Lily! Long time no see.
Lily: Hi Alex! Yeah, it’s been a while.
They sit and talk about old memories.

Lily: Do you still play the guitar?
Alex: Of course, every weekend.
The barista interrupts their conversation.
"""


In [25]:
def parse_story_simple(story_text):
    scenes = []
    current_scene = {"description": "", "dialogues": []}

    lines = [line.strip() for line in story_text.strip().split('\n') if line.strip()]

    for line in lines:
        if ':' in line and line.split(':', 1)[0].isalpha():
            speaker, speech = line.split(':', 1)
            current_scene["dialogues"].append({
                "character": speaker.strip(),
                "line": speech.strip()
            })
        else:
            if current_scene["description"] or current_scene["dialogues"]:
                scenes.append(current_scene)
                current_scene = {"description": "", "dialogues": []}
            current_scene["description"] = line.strip()

    if current_scene["description"] or current_scene["dialogues"]:
        scenes.append(current_scene)

    return scenes


Testing it

In [26]:
scenes = parse_story_simple(story)
for i, scene in enumerate(scenes):
    print(f"\n📘 Scene {i+1}")
    print("📝 Description:", scene['description'])
    print("💬 Dialogues:")
    for d in scene['dialogues']:
        print(f"  - {d['character']}: {d['line']}")



📘 Scene 1
📝 Description: Alex walks into the café and sees Lily.
💬 Dialogues:
  - Alex: Hey Lily! Long time no see.
  - Lily: Hi Alex! Yeah, it’s been a while.

📘 Scene 2
📝 Description: They sit and talk about old memories.
💬 Dialogues:
  - Lily: Do you still play the guitar?
  - Alex: Of course, every weekend.

📘 Scene 3
📝 Description: The barista interrupts their conversation.
💬 Dialogues:


In [27]:
# Define consistent visual styles per character
character_prompts = {
    "Alex": "young man with spiky hair wearing a red hoodie, comic book style",
    "Lily": "young woman with long black hair and a blue dress, comic book style",
    "Barista": "middle-aged man with an apron and a friendly smile, comic book style"
}


In [28]:
def get_style_prompt(style="comic"):
    style_dict = {
        "comic": "comic book, inked lines, halftone shadow",
        "anime": "anime style, colorful, clean lines",
        "pencil": "pencil sketch, hand drawn, grayscale"
    }
    return style_dict.get(style, "")


In [29]:
from diffusers import StableDiffusionPipeline
import torch

# Load the model (first time might take a few minutes)
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
pipe = pipe.to("cuda")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [9]:
def generate_image(scene_description, characters, style="comic", filename="scene.png"):
    char_prompt = ", ".join([character_prompts.get(c, "") for c in characters if c in character_prompts])
    prompt = f"{scene_description}, {char_prompt}, {get_style_prompt(style)}"
    image = pipe(prompt).images[0]
    image.save(filename)
    return filename


In [13]:
def add_dialog(image_path, dialogues, output_path="panel.png"):
    img = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(img)
    font = ImageFont.load_default()

    y_text = 10
    for dialog in dialogues:
        text = f"{dialog['character']}: {dialog['line']}"
        draw.text((10, y_text), text, font=font, fill="black")
        y_text += 30

    img.save(output_path)
    return output_path


In [12]:
def generate_comic(story_text, output_dir="comic_output", style="comic"):
    os.makedirs(output_dir, exist_ok=True)
    scenes = parse_story_simple(story_text)
    comic_panels = []

    for i, scene in enumerate(scenes):
        characters = list({d['character'] for d in scene['dialogues']})
        desc = scene['description']
        raw_img = generate_image(desc, characters, style=style, filename=f"{output_dir}/scene_{i}.png")
        final_img = add_dialog(raw_img, scene['dialogues'], output_path=f"{output_dir}/panel_{i}.png")
        comic_panels.append(final_img)

    return comic_panels


In [14]:
def create_comic_page(panels, grid_size=(2, 2), output_file="comic_page.png"):
    imgs = [Image.open(p) for p in panels]
    w, h = imgs[0].size
    grid_w, grid_h = grid_size

    comic_page = Image.new("RGB", (grid_w * w, grid_h * h), color="white")

    for index, img in enumerate(imgs):
        x = (index % grid_w) * w
        y = (index // grid_w) * h
        comic_page.paste(img, (x, y))

    comic_page.save(output_file)
    return output_file


In [15]:
story = """
Alex walks into the café and sees Lily.
Alex: Hey Lily! Long time no see.
Lily: Hi Alex! Yeah, it’s been a while.
They sit and talk about old memories.

Lily: Do you still play the guitar?
Alex: Of course, every weekend.
The barista interrupts their conversation.
"""

panels = generate_comic(story, style="comic")
create_comic_page(panels, grid_size=(2, 2), output_file="final_comic_page.png")


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

'final_comic_page.png'

In [16]:
sample_stories = [
    "Alex enters the forest and hears strange whispers...\nAlex: What was that?\nLily: I heard it too!",
    "Lily opens a magical book.\nLily: Wow, this looks ancient.\nAlex: Be careful with that!",
    # ... Add 8 more
]

for idx, s in enumerate(sample_stories):
    panels = generate_comic(s, output_dir=f"comic_output_{idx}")
    create_comic_page(panels, output_file=f"comic_page_{idx}.png")


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [17]:
from diffusers import StableDiffusionPipeline
import torch

# Use float32 for CPU-based generation
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float32)
pipe = pipe.to("cpu")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [18]:
sample_stories = [
    "Lily opens a magical door.\nLily: What is this place?\nAlex: It's... beautiful.",
    "Alex sees a strange figure in the alley.\nAlex: Who are you?\nFigure: Someone who knows too much.",
    "Lily finds a mysterious book.\nLily: I think this belonged to a wizard.",
    "Alex and Lily are on a rooftop at sunset.\nAlex: It’s perfect up here.\nLily: Just like old times.",
    "A portal opens in the sky.\nLily: We’re too late!\nAlex: Get ready to jump!",
    "They argue during a storm.\nLily: You never told me the truth!\nAlex: I was trying to protect you.",
    "They hide from patrolling robots.\nAlex: Stay low.\nLily: They’re scanning everything.",
    "They meet a talking cat.\nCat: Welcome to the city of mirrors.\nLily: Did… that cat just talk?",
    "In a forest glowing at night.\nLily: This place feels alive.\nAlex: It’s full of magic.",
    "They sit by a campfire after the battle.\nAlex: We survived.\nLily: But at what cost?"
]


In [30]:
styles = ["comic", "anime", "pencil"]
desc = "Alex and Lily standing on a cliff, watching a dragon in the sky."
chars = ["Alex", "Lily"]

from PIL import Image

style_imgs = []
for style in styles:
    img_path = generate_image(desc, chars, style=style, filename=f"comics/style_{style}.png")
    style_imgs.append(Image.open(img_path))

# Combine and save
style_guide = Image.new("RGB", (len(style_imgs) * 512, 512), "white")
for i, img in enumerate(style_imgs):
    style_guide.paste(img, (i * 512, 0))

style_guide.save("style_guide.png")


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [31]:
test_scenes = [
    "Alex reading a newspaper in a cafe",
    "Alex walking in the rain",
    "Alex running from a shadow"
]

char_consistency = []

for i, scene in enumerate(test_scenes):
    img_path = generate_image(scene, ["Alex"], filename=f"comics/consistency_alex_{i}.png")
    char_consistency.append(Image.open(img_path))

# Combine
consistency_panel = Image.new("RGB", (len(char_consistency) * 512, 512), "white")
for i, img in enumerate(char_consistency):
    consistency_panel.paste(img, (i * 512, 0))

consistency_panel.save("character_consistency.png")


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [44]:
import os
from PIL import Image

# Verify file path
image_path = "/content/final_comic_page.png"
print("✅ Exists:", os.path.exists(image_path))  # Should print True

# Define the layout function
def create_multi_panel_layout(image_paths, layout="2x2", output_path="/content/final_comic_page.png", padding=10, bg_color="white"):
    cols, rows = map(int, layout.split("x"))
    images = [Image.open(p).convert("RGBA") for p in image_paths if os.path.exists(p)]

    if not images:
        raise ValueError("❌ No valid images found. Please check file paths.")

    min_width = min(img.width for img in images)
    min_height = min(img.height for img in images)
    resized = [img.resize((min_width, min_height)) for img in images]

    grid_width = cols * min_width + (cols + 1) * padding
    grid_height = rows * min_height + (rows + 1) * padding
    comic_grid = Image.new("RGBA", (grid_width, grid_height), color=bg_color)

    for idx, img in enumerate(resized):
        if idx >= cols * rows:
            break
        col = idx % cols
        row = idx // cols
        x = padding + col * (min_width + padding)
        y = padding + row * (min_height + padding)
        comic_grid.paste(img, (x, y))

    comic_grid.save(output_path)
    print(f"✅ Comic saved at: {output_path}")
    return comic_grid

# Use the correct verified path
create_multi_panel_layout(
    [image_path] * 4,
    layout="2x2",
    output_path="/content/final_comic_page.png"
).show()


✅ Exists: True
✅ Comic saved at: /content/final_comic_page.png


In [45]:
from PIL import Image, ImageDraw, ImageFont

def create_character_consistency_demo(character_images, labels, output_path="/content/final_comic_page.png", spacing=20, font_size=24):
    images = [Image.open(img_path).convert("RGBA") for img_path in character_images]
    widths, heights = zip(*(img.size for img in images))

    total_width = sum(widths) + spacing * (len(images) - 1)
    max_height = max(heights) + font_size + 10

    combined = Image.new("RGBA", (total_width, max_height), (255, 255, 255, 255))
    draw = ImageDraw.Draw(combined)

    try:
        font = ImageFont.truetype("arial.ttf", font_size)
    except:
        font = ImageFont.load_default()

    x_offset = 0
    for i, img in enumerate(images):
        combined.paste(img, (x_offset, 0))
        draw.text((x_offset, img.height + 5), labels[i], fill="black", font=font)
        x_offset += img.width + spacing

    combined.save(output_path)
    return combined

# Example use
create_character_consistency_demo(
    ["/content/final_comic_page.png"] * 3,
    ["Scene 1", "Scene 2", "Scene 3"]
).show()


In [47]:
style_guide_content = """
Comic Strip Generator – Style Guide
===================================

🎨 Art Style Variations:
- Noir sketch (black and white, high contrast)
- Manga style (clean lines, exaggerated expressions)
- Watercolor fantasy (soft colors, painted look)

🎨 Color Schemes:
- Pastel for romantic/light scenes
- Monochrome for dramatic/serious scenes
- Neon/high contrast for sci-fi/action

🔤 Font Choices:
- Comic Sans MS for light-hearted panels
- Bangers/Action Man for action/sound FX
- Open Sans or Arial for neutral narrative/dialog

📐 Panel Layouts:
- 2x2 grid for short dialogues or story fragments
- 3x1 horizontal strip for fast-paced action
- 1x3 vertical scroll for mobile/web format

📌 Character Consistency:
- Use prompt templates with same keywords ("Alex, teen adventurer, brown hair, goggles")
- Maintain proportions and face shape per character

✅ Tips:
- Use consistent lighting and background elements
- Repeat key costume features
- Use same font per scene
"""
with open("/content/style_guide.png", "w") as f:
    f.write(style_guide_content)


In [49]:
from datetime import datetime

readme_content = f"""
# Comic Strip Generator 📚🎨

This project generates comic strips from story prompts using AI image models and layout algorithms.

## Features Implemented
✅ Script parsing
✅ Scene breakdown
✅ Character consistency
✅ Dialog extraction
✅ Image generation (Stable Diffusion-compatible)
✅ Panel layout (2x2, 3x1, etc.)
✅ Speech bubble placement
✅ Style customization (color, font, layout)
✅ Character consistency strip
✅ Multi-panel composition

## How to Use
1. Upload story prompt or comic scene
2. Choose art style, layout, font
3. Generate each panel
4. Assemble final comic with layout function
5. Add speech bubbles
6. Save comic to image or ZIP

## Output Samples
- `style_comic.png`: Original character panel
- `comic_with_speechbubble.png`: With dialog bubble
- `multipanel_output.png`: 2x2 layout
- `character_consistency.png`: Visual consistency
- `style_guide.txt`: Art + formatting guidelines

## Tools Used
- Python 3.8+
- PIL (Pillow)
- Diffusers / Stable Diffusion
- Google Colab

*Generated on: {datetime.now().strftime("%Y-%m-%d %H:%M")}, ready for submission.*
"""
with open("/content/style_guide.png", "w") as f:
    f.write(readme_content)


In [52]:
from google.colab import files

print("📂 Upload your .ipynb notebook now")
uploaded = files.upload()


📂 Upload your .ipynb notebook now


Saving ComicStripCraft_AI_Powered_Comic_Strip_Generator.ipynb to ComicStripCraft_AI_Powered_Comic_Strip_Generator.ipynb


In [53]:
import nbformat

notebook_path = "/content/ComicStripCraft_AI_Powered_Comic_Strip_Generator.ipynb"

with open(notebook_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# Remove widget metadata safely
if "widgets" in nb.metadata:
    del nb.metadata["widgets"]

# Save cleaned notebook
with open(notebook_path, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print("✅ Cleaned notebook metadata. You can now upload it to GitHub.")


✅ Cleaned notebook metadata. You can now upload it to GitHub.
